In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as f
from pathlib import Path
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
import os
from torchsummary import torchsummary
from PIL import Image

In [2]:
# Local

ORIGINAL_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'

In [ ]:
# On Server

ORIGINAL_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'

In [3]:
class NDIDatasetContrastiveLearning(Dataset):
    def __init__(self):
        super(NDIDatasetContrastiveLearning, self).__init__()
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        origins, targets = [], []
        to_tensor_func = torchvision.transforms.ToTensor()
        for original_image in original_images:
            origins.append(to_tensor_func(Image.open(str(original_image))))
            targets.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))))
        random_index = np.random.permutation(len(origins))
        self.origins, self.targets = [], []
        for index in random_index:
            self.origins.append(origins[index])
            self.targets.append(targets[index])

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx]

    def __len__(self):
        return len(self.origins)

In [4]:
from moco_model import MoCo


model = MoCo(torchvision.models.resnet18, dim=128, K=48)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)

In [11]:
# torchsummary.summary(model, input_size=[(1, 200, 200), (1, 200, 200)], batch_size=-1, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [5]:
def train_moco(net, criterion, optimizer, epochs, device):
    train_iter = DataLoader(NDIDatasetContrastiveLearning(), batch_size=8, shuffle=True, drop_last=True)
    model.cuda(device)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for origin, target in train_iter:
            origin, target = origin.cuda(device), target.cuda(device)
            output, labels = net(origin, target)
            loss = criterion(output, labels)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss {total_loss / len(train_iter)}')

In [6]:
train_moco(model, criterion, optimizer, 50, device)

Epoch 1, Loss 2.597166815661088
Epoch 2, Loss 3.8515222867329917
Epoch 3, Loss 3.7494990825653076
Epoch 4, Loss 3.6414151191711426
Epoch 5, Loss 3.4233253796895347
Epoch 6, Loss 3.05609925587972
Epoch 7, Loss 2.987532377243042
Epoch 8, Loss 3.2970497210820517
Epoch 9, Loss 2.9450793266296387
Epoch 10, Loss 2.9143287340799966
Epoch 11, Loss 2.7671069304148355
Epoch 12, Loss 2.736554225285848
Epoch 13, Loss 2.7076069513956704
Epoch 14, Loss 2.544517914454142
Epoch 15, Loss 2.499090234438578
Epoch 16, Loss 2.4189248085021973
Epoch 17, Loss 2.345860163370768
Epoch 18, Loss 2.284977078437805
Epoch 19, Loss 2.2567734718322754
Epoch 20, Loss 2.132940173149109
Epoch 21, Loss 2.1478540500005088
Epoch 22, Loss 2.12397966782252
Epoch 23, Loss 2.021037002404531
Epoch 24, Loss 2.0440903306007385
Epoch 25, Loss 1.9533389409383137
Epoch 26, Loss 2.019983728726705
Epoch 27, Loss 1.914332429567973
Epoch 28, Loss 1.9168410698572795
Epoch 29, Loss 1.8558295170466106
Epoch 30, Loss 1.8642888863881428
Epoc

In [7]:
def image_pair_matching(net, original_image, matching_image):
    net.cpu()
    net.eval()
    q = net.encoder_q(original_image)
    q = f.normalize(q, dim=1)
    k = net.encoder_k(matching_image)
    k = f.normalize(k, dim=1)
    logits = torch.mm(q, k.T)
    return logits

In [8]:
# Try to take the size of queue less than K, success

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_images_candidates = np.random.choice(target_images, 40)
target_tensor = []
for i in range(20):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))).unsqueeze(0))
for i in range(20, 40):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
image_pair_matching(model, original_tensor, target_tensor)

tensor([[0.1602, 0.1168, 0.1395, 0.1161, 0.2010, 0.1194, 0.0940, 0.0536, 0.1289,
         0.1064, 0.1281, 0.1714, 0.1407, 0.1310, 0.1047, 0.1310, 0.1194, 0.2436,
         0.1281, 0.1409, 0.3409, 0.0951, 0.1625, 0.1391, 0.1456, 0.1388, 0.1216,
         0.2257, 0.1398, 0.2367, 0.1064, 0.2324, 0.1189, 0.1511, 0.1047, 0.1310,
         0.1395, 0.0883, 0.0883, 0.1884, 0.1624]], grad_fn=<MmBackward>)

In [11]:
# Try to take all matching images

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_tensor = []
for i in range(1, 55):
    target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
print(str(original_image.name).split('_')[0])

2


In [12]:
logits = image_pair_matching(model, original_tensor, target_tensor)
print(logits)
print(torch.argmax(logits))

tensor([[0.1598, 0.3709, 0.1985, 0.1398, 0.0803, 0.1123, 0.1783, 0.1746, 0.1006,
         0.1923, 0.2008, 0.1236, 0.1180, 0.2244, 0.1906, 0.1739, 0.1598, 0.2098,
         0.0587, 0.1832, 0.1987, 0.1745, 0.0828, 0.1505, 0.2023, 0.2023, 0.1683,
         0.0951, 0.1741, 0.1218, 0.1286, 0.0872, 0.1532, 0.1567, 0.1228, 0.2039,
         0.0884, 0.2117, 0.2043, 0.2047, 0.1642, 0.1117, 0.2014, 0.1685, 0.1717,
         0.1340, 0.1512, 0.1313, 0.1718, 0.1934, 0.1324, 0.1725, 0.1725, 0.1651]],
       grad_fn=<MmBackward>)
tensor(1)
